# 🥋 Lekcja 33: PyTorch 2.0 & torch.compile (Darmowe Przyspieszenie)

To największa zmiana w historii PyTorcha.
Funkcja `torch.compile(model)` analizuje Twój kod, znajduje sekwencje operacji, które można połączyć, i generuje w locie **zoptymalizowany kod C++/Triton**.

**Główne zalety:**
1.  **Kernel Fusion:** Łączenie małych operacji (Add, Mul, Gelu) w jeden duży kernel CUDA. Mniej czytania/pisania do VRAM.
2.  **Graph Capture:** Eliminuje narzut Pythona (Python Overhead).

**Tryby kompilacji:**
*   `default`: Balans między szybkością kompilacji a szybkością działania.
*   `reduce-overhead`: Używa CUDA Graphs. Dobre dla małych batchy.
*   `max-autotune`: Najdłużej się kompiluje (profiluje różne konfiguracje), ale daje najszybszy kod.

*Uwaga: Na Windowsie torch.compile wciąż bywa kapryśne (często wymaga MSVC lub działa wolniej). Na Linuxie/WSL to rakieta.*

In [1]:
import torch
import torch._dynamo
import time

# Sprawdźmy, czy mamy GPU (Ampere lub nowsze najlepiej korzystają z Tritona)
if torch.cuda.is_available():
    device = "cuda"
    # Resetujemy ustawienia dynamo (dla czystości testu)
    torch._dynamo.reset()
else:
    device = "cpu"
    print("⚠️ Testujemy na CPU. Przyspieszenie będzie mniejsze niż na GPU.")

print(f"Urządzenie: {device}")

Urządzenie: cuda


## Model Testowy (Element-wise Heavy)

Kompilator błyszczy tam, gdzie jest dużo operacji **element-wise** (działających na każdym pikselu osobno), np. funkcje aktywacji, normalizacje, dodawanie.
Zbudujemy sieć, która robi dużo "matematycznej drobnicy".

In [2]:
class HeavyOpsLayer(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.w = torch.nn.Linear(1024, 1024)

    def forward(self, x):
        # Sekwencja operacji punktowych (Element-wise)
        # W Eager Mode każda z nich to osobny kernel CUDA (osobny zapis/odczyt pamięci).
        x = self.w(x)
        x = torch.cos(x)
        x = torch.sin(x)
        x = torch.relu(x)
        x = x * 2
        x = x + 1
        return x

model = HeavyOpsLayer().to(device)
input_data = torch.randn(2048, 1024).to(device) # Spory batch

print("Model gotowy.")

Model gotowy.


## Benchmark: Eager Mode (Standard)

Zmierzmy czas w "starym" PyTorch.
Używamy `torch.cuda.synchronize()`, żeby zmierzyć prawdziwy czas wykonania na GPU (bo GPU działa asynchronicznie).

In [3]:
def benchmark(model, x, runs=100):
    # Rozgrzewka (Warmup)
    for _ in range(10):
        _ = model(x)
    
    if device == "cuda":
        torch.cuda.synchronize()
        
    start = time.time()
    for _ in range(runs):
        _ = model(x)
        
    if device == "cuda":
        torch.cuda.synchronize()
        
    end = time.time()
    return (end - start) / runs

# Test Eager
eager_time = benchmark(model, input_data)
print(f"Czas Eager Mode: {eager_time*1000:.3f} ms / iter")

Czas Eager Mode: 0.752 ms / iter


## Benchmark: Compiled Mode

Teraz magia. `torch.compile`.

**Ważne:** Pierwsze uruchomienie skompilowanego modelu będzie **bardzo wolne**.
Dlaczego? Bo wtedy trwa kompilacja (JIT - Just In Time). PyTorch analizuje kod, generuje Triton/C++ i kompiluje go.
Dopiero drugie i kolejne uruchomienia są szybkie.

In [5]:
import os
import sys

print("Kompilowanie modelu (to może chwilę potrwać)...")

# Wybór backendu w zależności od systemu
# Na Linuxie 'inductor' (Triton) jest domyślny i najszybszy.
# Na Windowsie Triton nie działa, więc próbujemy 'cudagraphs' lub 'eager' (brak optymalizacji, ale testuje API).
if os.name == 'nt': # Windows
    print("ℹ️ Wykryto Windows. Triton nie jest dostępny.")
    print("Próba użycia backendu 'cudagraphs' (wymaga sterowników) lub 'eager'...")
    preferred_backend = "eager" # Bezpieczny fallback, żeby kod przeszedł
else:
    preferred_backend = "inductor" # Domyślny na Linux

try:
    # Uruchamiamy kompilację
    compiled_model = torch.compile(model, backend=preferred_backend)
    
    # Pierwsze uruchomienie (Kompilacja/Rozgrzewka)
    start_compile = time.time()
    _ = compiled_model(input_data)
    print(f"✅ Pierwsze uruchomienie (Warmup): {time.time() - start_compile:.2f} s")

    # Właściwy Benchmark
    compiled_time = benchmark(compiled_model, input_data)
    print(f"Czas Compiled Mode ({preferred_backend}): {compiled_time*1000:.3f} ms / iter")

    # Podsumowanie
    speedup = eager_time / compiled_time
    print(f"\n🚀 Przyspieszenie: {speedup:.2f}x")
    
    if preferred_backend == "eager":
        print("(Uwaga: Backend 'eager' nie optymalizuje kodu, służy tylko do testu API na Windowsie)")

except Exception as e:
    print(f"\n⚠️ Nawet fallback zawiódł: {e}")
    print("To normalne na Windowsie bez pełnego środowiska C++.")
    print("Lekcja zaliczona: Wiesz jak używać torch.compile, ale potrzebujesz Linuxa, żeby zobaczyć zysk.")

Kompilowanie modelu (to może chwilę potrwać)...
ℹ️ Wykryto Windows. Triton nie jest dostępny.
Próba użycia backendu 'cudagraphs' (wymaga sterowników) lub 'eager'...
✅ Pierwsze uruchomienie (Warmup): 0.04 s
Czas Compiled Mode (eager): 0.937 ms / iter

🚀 Przyspieszenie: 0.80x
(Uwaga: Backend 'eager' nie optymalizuje kodu, służy tylko do testu API na Windowsie)


## 🥋 Black Belt Summary

1.  **Kernel Fusion:** To główny powód przyspieszenia. Zamiast:
    *   `Load -> Cos -> Store`
    *   `Load -> Sin -> Store`
    `torch.compile` robi:
    *   `Load -> Cos -> Sin -> Store` (w rejestrach procesora GPU).
    Oszczędzasz przepustowość pamięci (Memory Bandwidth), która jest wąskim gardłem.

2.  **Kiedy używać?**
    *   Zawsze na produkcji (Inference).
    *   Zawsze przy treningu dużych modeli (Transformerów).

3.  **Wymagania:**
    *   Nowoczesne GPU (Turing, Ampere).
    *   Na Windowsie może wymagać instalacji `Microsoft C++ Build Tools`. Na Linuxie działa "out of the box".
    *   Kod nie może być zbyt dynamiczny (skomplikowane `if/else` zależne od danych mogą powodować "Graph Breaks", co psuje optymalizację).